In [ ]:
#Classical Way

# #note: args Setting catalog
# path  
# name
# RADIUS_SMOOTH
# scale_value
# goodFeaturesToTrack(...)

# cv2.calcOpticalFlowPyrLK(...)
# cv2.goodFeaturesToTrack(...)

# "direction":
# # out = cv2.VideoWriter(path+file_name+'_out_'+date_time+'.avi', fourcc, fps, (w * 3, h))
# out = cv2.VideoWriter(path+file_name+'_out_'+date_time+'.avi', fourcc, fps, (w, h * 3))

# # frame_out = cv2.hconcat([frame_circle,frame, frame_stabilized_original])
#   frame_out = cv2.vconcat([frame_circle,frame, frame_stabilized_original])




import numpy as np
import cv2



def movementAverage(curve, radius): 
  window_size = 2 * radius + 1
  # Set a filter filter
  filter = np.ones(window_size)/window_size 
  # Add padding
  curve_pad = np.lib.pad(curve, (radius, radius), 'edge') 
  # Convolute 
  curve_smooth = np.convolve(curve_pad, filter, mode='same') 
  # Delete padding 
  curve_smooth = curve_smooth[radius:-radius]
  # Return smoothed curve
  return curve_smooth 

def getSmooth(trajectory,radius_smooth_value): 
  trajectory_smooth = np.copy(trajectory) 
  # Filter x, y and angle curves
  for i in range(3):
    trajectory_smooth[:,i] = movementAverage(trajectory[:,i], radius = radius_smooth_value)
  return trajectory_smooth

def scaleImage(frame,scale_value):
  fs = frame.shape
  # Scale the frame image to scale_value with fixed center
  #Transformation matrix
  Trans_Matrix = cv2.getRotationMatrix2D((fs[1]/2, fs[0]/2), 0, scale_value)
  # cv2.warpAffine(img,M,(rows,cols))
  frame = cv2.warpAffine(frame, Trans_Matrix, (fs[1], fs[0]))
  return frame

# Read input video
path = './drive/MyDrive/ColabFiles/IVPProject/'
# file_name = 'video.mp4'
# file_name = 'IMG_4015.mp4'
# file_name = 'IMG_4019.MOV'
# file_name = 'IMG_4020.MOV' 
# file_name = 'Underwater.mp4'
# file_name = 'IMG_4067.MOV'

#test three videos
# file_name = 'Forward.mp4'
# file_name = 'Still.MOV'
file_name = 'Random.MOV'
file_name = 'Deep_learning_video.mp4'
cap = cv2.VideoCapture(path+file_name) 

 
# Get the frame count
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
 
# Get the width and height
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get frames per second
fps = cap.get(cv2.CAP_PROP_FPS)


# Read frame from the first image
_, prev = cap.read() 
 
# Convert frame to grayscale
previous_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY) 

# Set transformation matrix
transforms = np.zeros((num_frames-1, 3), np.float32) 

# Save key points as a list
all_keypoints = list()

# Attention: except first frame, so "-2"
for i in range(num_frames-2):
  # Detect feature points in previous frame
  previous_points = cv2.goodFeaturesToTrack(previous_gray,
                                     maxCorners=200,
                                     qualityLevel=0.02,
                                     minDistance=20,
                                     blockSize=3)
  #Save valid key points
  single_image_keypoints = list()
  if previous_points is not None and len(previous_points)>0:
        for x,y in np.float32(previous_points).reshape(-1,2):
            single_image_keypoints.append((x,y))
  all_keypoints.append((single_image_keypoints))

  
  # Read next frame
  success, curr = cap.read() 
  
  if not success: 
    break 

  # Convert to grayscale
  current_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY) 

  # Calculate optical flow
  current_points, status, err = cv2.calcOpticalFlowPyrLK(previous_gray, current_gray, previous_points, None) 

  
  # Select valid points
  idx = np.where(status==1)[0]
  previous_points = previous_points[idx]
  current_points = current_points[idx]

  # Find transformation matrix
  m,inlier = cv2.estimateAffine2D(previous_points, current_points)

  # Extract the translation
  dx = m[0,2]
  dy = m[1,2]

  # Extract the rotation angle
  da = np.arctan2(m[1,0], m[0,0])
   
  # Save transformation
  transforms[i] = [dx,dy,da]
   
  # Forward to next frame
  previous_gray = current_gray

  print("Frame Process: " + str(i) +  "/" + str(num_frames) + " - Points : " + str(len(previous_points)))

# Compute trajectory
trajectory = np.cumsum(transforms, axis=0) 
 
# Large -> stable,
# Small -> sensitive to quick movement such as a sudden panning
# RADIUS_SMOOTH=50 
# RADIUS_SMOOTH=100
RADIUS_SMOOTH=1000

# Calculate smoothed trajectory
trajectory_smooth = getSmooth(trajectory,RADIUS_SMOOTH) 

# Calculate difference between trajectory_smooth and trajectory
difference = trajectory_smooth - trajectory
 
# Calculate newer transformation array
transformation_smooth = transforms + difference




Frame Process: 0/486 - Points : 200
Frame Process: 1/486 - Points : 200
Frame Process: 2/486 - Points : 200
Frame Process: 3/486 - Points : 200
Frame Process: 4/486 - Points : 200
Frame Process: 5/486 - Points : 200
Frame Process: 6/486 - Points : 199
Frame Process: 7/486 - Points : 199
Frame Process: 8/486 - Points : 200
Frame Process: 9/486 - Points : 200
Frame Process: 10/486 - Points : 200
Frame Process: 11/486 - Points : 200
Frame Process: 12/486 - Points : 200
Frame Process: 13/486 - Points : 200
Frame Process: 14/486 - Points : 200
Frame Process: 15/486 - Points : 200
Frame Process: 16/486 - Points : 200
Frame Process: 17/486 - Points : 200
Frame Process: 18/486 - Points : 200
Frame Process: 19/486 - Points : 200
Frame Process: 20/486 - Points : 200
Frame Process: 21/486 - Points : 200
Frame Process: 22/486 - Points : 200
Frame Process: 23/486 - Points : 200
Frame Process: 24/486 - Points : 200
Frame Process: 25/486 - Points : 200
Frame Process: 26/486 - Points : 200
Frame Proce

In [ ]:
# all_keypoints[0]
# w

1920

In [ ]:



# Set the four character code for output
# In Python, "*" could help divide the string and send the args one by one
fourcc = cv2.VideoWriter_fourcc(*'MJPG')

import time

# Set time format to 2016-03-20 11:22:33
date_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) 
# out = cv2.VideoWriter(path+file_name+'_out_'+date_time+'.avi', fourcc, fps, (w * 3, h))
out = cv2.VideoWriter(path+file_name+'_out_'+date_time+'.avi', fourcc, fps, (w, h * 3))


# Get back to first frame in the stream
cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 

# Write num_frames-1 transformed frames
for i in range(num_frames-2):
  # Read next frame
  success, frame = cap.read() 
  if not success:
    break

  


  # Extract transformations from the new transformation matrix
  dx = transformation_smooth[i,0]
  dy = transformation_smooth[i,1]
  da = transformation_smooth[i,2]

  # Reconstruct transformation matrix
  m = np.zeros((2,3), np.float32)
  m[0,0] = np.cos(da)
  m[0,1] = -np.sin(da)
  m[1,0] = np.sin(da)
  m[1,1] = np.cos(da)
  m[0,2] = dx
  m[1,2] = dy

  # Apply affine wrapping
  frame_stabilized = cv2.warpAffine(frame, m, (w,h))

  # Scale border artifacts
  SCALE_VALUE = 1.2
  frame_stabilized = scaleImage(frame_stabilized,SCALE_VALUE) 


  # Write the frame to the file
  
  import copy

  # Display 3 colors and 2 color
  # frame[:,:,0]=0
  # frame[:,:,1]=0
  frame_stabilized_original = copy.deepcopy(frame_stabilized)
  # frame_stabilized[:,:,1]=0
  frame_stabilized[:,:,2]=0
  
  frame_circle = frame[:]

  # Show key points
  local_keypoints = all_keypoints[i]
  if local_keypoints is not None and len(local_keypoints)>0:
    for x,y in local_keypoints:
      cv2.circle(frame_circle, (int(x),int(y)), 5, (255,255,0))

  # Make it transparent and set a overlay view.
  ALPHA = 0.62
  frame = ((1-ALPHA) * frame.astype(np.float) + ALPHA * frame_stabilized.astype(np.float)).astype(np.uint8)

  # frame_out = cv2.hconcat([frame_circle,frame, frame_stabilized_original])
  frame_out = cv2.vconcat([frame_circle,frame, frame_stabilized_original])

  
  # res = cv2.addWeighted(img_cat, 0.8, img_dog_1, 0.6, 0)  # 权重 权重 偏置值


  # # Resize large image
  # if(frame_out.shape[1] > 2000): 
  #   frame_out = cv2.resize(frame_out, (frame_out.shape[1]//2, frame_out.shape[0]//2));
  
  # from google.colab.patches import cv2_imshow
  # cv2.imshow("Before and After", frame_out)
  cv2.waitKey(10)
  out.write(frame_out)

# Release
# cap.release()
out.release()

# Close
# cv2.destroyAllWindows()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
